In [1]:
# Install once only - python official library from AWS
# !pip install boto3

### Define the Configuration

In [2]:
aws_accesskey = <Your Access Key>
aws_secretaccess = <Your secret access key>
myregion= <your region>

In [3]:
import boto3

In [4]:
import logging

In [5]:
from botocore.exceptions import ClientError

In [6]:
import base64

In [7]:
import json

In [8]:
import pprint

In [9]:
import boto3
import base64
import json
import pprint

# Initialize S3 and Lambda clients
S3_client = boto3.client('s3', aws_access_key_id=aws_accesskey, aws_secret_access_key=aws_secretaccess, region_name=myregion)
lambda_client = boto3.client('lambda', aws_access_key_id=aws_accesskey, aws_secret_access_key=aws_secretaccess, region_name=myregion)

# Replace with the name of your Lambda function
function_name = 'mylambda_function'

def analyze_images_local(aws_access, aws_secret, aws_region, img_path):
    print("Analyzing local image:")
    
    with open(img_path, 'rb') as image_file:
        image_bytes = image_file.read()
        data = base64.b64encode(image_bytes).decode("utf8")
        
        lambda_payload = {"image": data}
        
        # Invoking the lambda function with the event payload
        response = lambda_client.invoke(
            FunctionName=function_name,
            Payload=json.dumps(lambda_payload))
        decoded = json.loads(response['Payload'].read().decode())
        pprint.pprint(decoded)

In [10]:
# # If you are using a file from the local computer, you need supply the file path
image_path = <image file path>

In [11]:
analyze_images_local(aws_accesskey,aws_secretaccess,myregion,<image file path>)

Analyzing local image:
{'body': '{"Labels": [{"Name": "Landscape", "Confidence": 99.9991455078125, '
         '"Instances": [], "Parents": [{"Name": "Nature"}, {"Name": '
         '"Outdoors"}], "Aliases": [], "Categories": [{"Name": "Nature and '
         'Outdoors"}]}, {"Name": "Nature", "Confidence": 99.9991455078125, '
         '"Instances": [], "Parents": [{"Name": "Outdoors"}], "Aliases": [], '
         '"Categories": [{"Name": "Nature and Outdoors"}]}, {"Name": '
         '"Outdoors", "Confidence": 99.9991455078125, "Instances": [], '
         '"Parents": [], "Aliases": [], "Categories": [{"Name": "Nature and '
         'Outdoors"}]}, {"Name": "Scenery", "Confidence": 99.99686431884766, '
         '"Instances": [], "Parents": [{"Name": "Nature"}, {"Name": '
         '"Outdoors"}], "Aliases": [], "Categories": [{"Name": "Nature and '
         'Outdoors"}]}, {"Name": "Sky", "Confidence": 99.98040771484375, '
         '"Instances": [], "Parents": [{"Name": "Nature"}, {"Name": '
   

In [24]:
S3_client = boto3.client('s3', aws_access_key_id=aws_accesskey, aws_secret_access_key=aws_secretaccess, region_name=myregion)
lambda_client = boto3.client('lambda', aws_access_key_id=aws_accesskey, aws_secret_access_key=aws_secretaccess, region_name=myregion)
function_name = 'mylambda_function'< Mention your Lambda Function >

bucket_name = <Your Bucket Name>
object_key = <Your Object Key>

def analyze_image_s3(aws_access,aws_secret,aws_region,bucket_name,object_key):
    print("Analyzing image in S3 bucket:")
    
    # Get the image data from S3
    response = S3_client.get_object(Bucket=bucket_name, Key=object_key)
    image_data = response['Body'].read()
    image_data_encoded = base64.b64encode(image_data).decode("utf-8")
    
    # Create Lambda event payload
    event = {
        'S3Bucket': bucket_name,
        'S3Object': object_key,
        'ImageBytes': image_data_encoded
    }
    
    # Invoke the Lambda function with the event payload
    response = lambda_client.invoke(
        FunctionName=function_name,
        InvocationType='RequestResponse',
        Payload=json.dumps(event)
    )
    
    decoded = json.loads(response['Payload'].read().decode())
    pprint.pprint(decoded)


In [25]:
analyze_image_s3(aws_accesskey,aws_secretaccess,myregion,bucket_name,object_key)

Analyzing image in S3 bucket:
{'body': {'Error': 'AccessDenied',
          'ErrorMessage': "Couldn't analyze image: Access Denied"},
 'statusCode': 400}
